# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [2]:
! hadoop fs -ls -h /data/

Found 2 items
-rw-r--r--   1 root supergroup     30.7 M 2023-09-24 20:38 /data/clickstream.csv
drwxr-xr-x   - root supergroup          0 2023-09-24 20:38 /data/transactions


In [130]:
import findspark
findspark.init()


In [229]:
import sys
import itertools

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import length
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.functions import lower

In [ ]:
sc = SparkContext(appName="sga")
se = SparkSession(sc)

# Spark SQL solution with <= 2 queries

In [238]:
df = se.read.csv("hdfs:///data/clickstream.csv", sep='\t', header=True)

In [239]:
df.registerTempTable('clickstream')

In [275]:
truncated = se.sql("""
select 
ROW_NUMBER() over (order by clickstream.session_id, clickstream.timestamp) as index,
clickstream.user_id,
clickstream.session_id,
clickstream.event_type,
clickstream.event_page,
clickstream.timestamp,
error_ts
from clickstream

left join
(select
    session_id,
    timestamp as error_ts
from 
    clickstream
where event_type like '%error%') as erorr_ts
on clickstream.session_id = erorr_ts.session_id
where clickstream.timestamp < error_ts
    """)

In [276]:
truncated.registerTempTable('truncated')

In [293]:
routes = se.sql("""
select 
session_id,
event_page
from
(SELECT
    *, 
    Lag(event_page, 1) OVER(order by index) AS lag_ep
FROM truncated) as lagged
where event_page != lag_ep;
  """).toPandas()

2023-10-02 07:55:53,715 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-10-02 07:55:53,715 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-10-02 07:55:53,715 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-10-02 07:55:54,605 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-10-02 07:55:54,605 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2023-10-02 07:55:55,144 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to 

In [295]:
route_count = {}

for si, f in routes.groupby('session_id'):
    route = '-'.join(f['event_page'].values)
    if route not in route_count:
        route_count[route] = 0
        
    route_count[route] += 1

In [296]:
with open('solution_df.txt', 'w') as sol:
    for r, c in sorted(route_count.items(), key=lambda x: x[1], reverse=True)[:30]:
        sol.write(f'{r}, {c}\n')

# Spark rdd solution

In [309]:
df = se.read.csv("hdfs:///data/clickstream.csv", sep='\t', header=True)

In [312]:
df.rdd.groupBy(lambda x: x['session_id']).collect()

[('54', <pyspark.resultiterable.ResultIterable at 0x7f8eeb43b910>),
 ('280', <pyspark.resultiterable.ResultIterable at 0x7f8eea1d4b80>),
 ('508', <pyspark.resultiterable.ResultIterable at 0x7f8ee6cb45e0>),
 ('348', <pyspark.resultiterable.ResultIterable at 0x7f8ee7781ba0>),
 ('190', <pyspark.resultiterable.ResultIterable at 0x7f8eebc0faf0>),
 ('552', <pyspark.resultiterable.ResultIterable at 0x7f8ee7881000>),
 ('744', <pyspark.resultiterable.ResultIterable at 0x7f8f63b4b8b0>),
 ('334', <pyspark.resultiterable.ResultIterable at 0x7f8f4a110c40>),
 ('446', <pyspark.resultiterable.ResultIterable at 0x7f8f63cb0a30>),
 ('86', <pyspark.resultiterable.ResultIterable at 0x7f8f5b0ab310>),
 ('435', <pyspark.resultiterable.ResultIterable at 0x7f8eed9fb6d0>),
 ('682', <pyspark.resultiterable.ResultIterable at 0x7f8ef4761db0>),
 ('209', <pyspark.resultiterable.ResultIterable at 0x7f8ef2ffe980>),
 ('102', <pyspark.resultiterable.ResultIterable at 0x7f8ef1062ec0>),
 ('596', <pyspark.resultiterable.Res

In [ ]:
route_counts = (
    cs.rdd
    .groupBy(lambda x: x['session_id'])
    .aggregate(lambda a, b: f'{a}-{b}')
    .collect()
)


# Spark df solution

In [297]:
import pyspark.sql.functions as f
df = se.read.csv("hdfs:///data/clickstream.csv", sep='\t', header=True)
# df.groupby("col1").agg(f.concat_ws(", ", f.collect_list(df.col2)))

In [298]:
df.limit(10).show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
+-------+----------+------------+----------+----------+



In [299]:
df = df.withColumn(
    "event_page_w_error", 
    when(
        lower(col('event_type')).contains('error'), concat(lit("error_"), col("event_page"))
    ).otherwise(col("event_page")))

In [300]:
grouped = df.groupBy("session_id").agg(f.concat_ws("-", f.collect_list(df['event_page_w_error'])).alias('route'))
# df.limit(10).show()

In [301]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

def clean_route(r):
    tokens = r.split('-')
    mark = len(tokens)
    # mark = 2
    for i, t in enumerate(tokens):
        if 'error' in t.lower():
            mark = i
            break
    
    tokens = tokens[:mark]
    tokens = [k for k, _g in itertools.groupby(tokens)]
    return '-'.join(tokens[:mark])
    
udf_clean_route = udf(lambda x: clean_route(x), StringType()) 
        

In [302]:
grouped = grouped.withColumn("non_error_route", udf_clean_route(grouped['route']))

In [303]:
grouped = grouped \
    .withColumn("len_route",length(col("route"))) \
    .withColumn("len_non_error_route",length(col("non_error_route")))

In [304]:
grouped.groupby('non_error_route').count().sort('count', ascending=False).toPandas()

,non_error_route,count
0,main,56
1,main-archive,13
2,main-internet,10
3,main-bonus,6
4,main-news,6
...,...,...
949,main-bonus-tariffs-archive-rabota-main-archive...,1
950,main-news-archive-bonus-news-archive-internet-...,1
951,main-bonus-digital-news-vklad-archive-main-arc...,1
952,main-bonus-main-tariffs-vklad-news-online-rabo...,1


In [305]:
df = cs.toPandas()

In [306]:
df[df['session_id'] == '0']

,user_id,session_id,event_type,event_page,timestamp
14082,2536,0,page,main,1695658516
14086,2536,0,page,tariffs,1695658533
14217,2536,0,page,online,1695659155
27100,412,0,page,main,1695726743
27163,412,0,event,main,1695727062
...,...,...,...,...,...
243732,4488,0,event,rabota,1696870352
243769,4488,0,event,rabota,1696870529
243847,4488,0,event,rabota,1696870940
243854,4488,0,page,internet,1696870972


In [307]:
route_count = {}

for si, f in df.groupby('session_id'):
    f = f.sort_values('timestamp').reset_index(drop=True)
    fe = f[f['event_type'].str.lower().str.contains('error')]
    if not fe.empty:
        # print(fe.loc[:, 'timestamp'])
        f = f[f['timestamp'] < fe.loc[:, 'timestamp'].values[0]]
    
    f = f[f['event_page'] != f['event_page'].shift(1)]
    route = '-'.join(f['event_page'].values)
    if route not in route_count:
        route_count[route] = 0
        
    route_count[route] += 1


In [308]:
with open('solution_df.txt', 'w') as sol:
    for r, c in sorted(route_count.items(), key=lambda x: x[1], reverse=True)[:30]:
        sol.write(f'{r}, {c}\n')
